# \[set-up\], imports
See README for more details about installing necessary packages and setting up your environment.

In [2]:
import pint_pal as pp
import pint_pal.report
import pint_pal.lite_utils as lu
import pint_pal.plot_utils as pu
from pint_pal.timingconfiguration import TimingConfiguration
from astropy import log
from pint.fitter import ConvergenceFailure
import pint.fitter
import os
import sys
import copy
from astropy.visualization import quantity_support
quantity_support()

# notebook gives interactive plots but not until the kernel is done
%matplotlib notebook
# inline gives non-interactive plots right away
#%matplotlib inline

# Set logging level (PINT uses loguru)
log.setLevel("INFO") # Set desired verbosity of log statements (DEBUG/INFO/WARNING/ERROR)
pint.logging.setup(level="WARNING", usecolors=True)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


1

In [4]:
hasnoise = False
if hasnoise == False:
    ext = '_prenoise'
else:
    ext = '_postnoise'

# develop/update timing solution

Load configuration (`.yaml`) file, get TOAs and timing model; if you're running from the root of the git distribution, simply edit the `.yaml` file name, otherwise include relevant paths to the `.yaml` file, and `.par`/`.tim` directories as kwargs (see commented example).

In [ ]:
config = "configs/[psr_name].[nb or wb].yaml"  # fill in actual path
par_directory = None   # default location
tim_directory = None   # default location
tc = TimingConfiguration(config, par_directory=par_directory, tim_directory=tim_directory)

# Summary report will be added to throughout the notebook & rendered at the end
report = pint_pal.report.Report(title=f"{tc.get_source()} PINT combination summary")
report.add_markdown("Software", f"Python version: {sys.version.split()[0]}")
report.add_markdown("Software", f"PINT version: {pint.__version__}")
report.add_markdown("Software", f"PINT Pal version: {pp.__version__}")

# To combine TOAs, assumption is that cuts have already been applied properly
mo,to = tc.get_model_and_toas(apply_initial_cuts=False,usepickle=False)

# Apply manual cuts
tc.manual_cuts(to)

# Computing pulse numbers ensures param changes in the model will not break phase connection
to.compute_pulse_numbers(mo)

# Set non-binary epochs to the center of the data span
lu.center_epochs(mo,to)

# Summarize TOAs present
summary = to.get_summary()
report.add_markdown("Summary of TOAs", f"```\n{summary}```")

In [ ]:
# Define the fitter object and plot pre-fit residuals
fo = tc.construct_fitter(to,mo)
pu.plot_residuals_time(fo, restype='prefit', legend=False)
report.add_plot("Prefit residuals")
if mo.is_binary:
    pu.plot_residuals_orb(fo, restype='prefit', legend=False)
    report.add_plot("Prefit residuals")

In [ ]:
# Set free params based on list in the config file (want to update JUMP handling differently soon)
fo.model.free_params = tc.get_free_params(fo)

# Do the fit
try:
    fo.fit_toas(maxiter=tc.get_niter())
    fo.model.CHI2.value = fo.resids.chi2
except ConvergenceFailure:
    run_Ftest = False
    log.warning('Failed to converge; moving on with best result, but should address before final version.')

In [ ]:
# Plot post-fit residuals, print summary of results, write prenoise solution
pu.plot_residuals_time(fo, restype='postfit', legend=False)
report.add_plot("Postfit residuals")
if mo.is_binary:
    pu.plot_residuals_orb(fo, restype='postfit', legend=False)
    report.add_plot("Postfit residuals")
    
summary = fo.get_summary()
report.add_markdown("Fit summary", f"```\n{summary}\n```")
lu.check_convergence(fo)

lu.write_par(fo,toatype=tc.get_toa_type(),addext=ext)

In [ ]:
# Look at the whitened residuals as well if noise analysis has been completed.
if hasnoise == True:
    pu.plot_residuals_time(fo, restype='postfit', whitened=True, avg=True, mixed_ecorr=True, legend=True)
    report.add_plot("Postfit whitened residuals")
    if mo.is_binary:
        pu.plot_residuals_orb(fo, restype='postfit', whitened=True, avg=True, mixed_ecorr=True, legend=True)
        report.add_plot("Postfit whitened residuals")

# F-test for additional parameters

Note: This is a time consuming section and should only be run if workign with data with a noise solution. 

In [ ]:

run_Ftest = False
Ftest_dict = None
if run_Ftest:
    savedLevel = log.getEffectiveLevel()
    try:
        log.setLevel("WARNING")
        Ftest_dict = run_Ftests(fo, 
                                alpha=0.0027, 
                                NITS=tc.get_niter())
    finally:
        log.setLevel(savedLevel)

report.add_markdown("F-test summary", f"```\n{Ftest_dict}\n```")

# generate \[summary\] report

In [ ]:
# Generate the PDF report
report.generate_pdf(f"{tc.get_source()}-pint-summary.pdf")